In [ ]:
pip install pyspark

     |████████████████████████████████| 281.3 MB 41 kB/s 
     |████████████████████████████████| 198 kB 55.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=f45f85b65c4f62e9f85f123b46fdb647c0882875753b82dbfec149b740f76cf9
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [ ]:
import sys
from pyspark.sql import SparkSession, functions, types
from pyspark.context import SparkContext
from pyspark.sql.types import IntegerType,StringType
from pyspark.sql.functions import *
from pyspark.sql.functions import isnan, when, count, col
#ML Algorithms
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler, SQLTransformer
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import LinearSVC
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
path_prefix = "/content/gdrive/MyDrive/cadorsdata"
import sys
sys.path.insert(1, path_prefix)

Mounted at /content/gdrive


In [ ]:
spark = SparkSession.builder.appName('ETL temp').getOrCreate()
# assert spark.version >= '3.0' # make sure we have Spark 3.0+
spark.sparkContext.setLogLevel('WARN')
sc = spark.sparkContext

In [ ]:
aircraft_df = spark.read.json('/content/gdrive/MyDrive/cadorsdata_aircraft/*')
weather_df = spark.read.json('/content/gdrive/MyDrive/cadorsdata_weather/*')

In [ ]:
aircraft_df = aircraft_df.na.drop(subset=["aircraft_category","make","operator_type","phase_of_flight","year_built"])

In [ ]:
print(aircraft_df.count())
print(aircraft_df.select(['CADORS Number']).distinct().count())

235425
212176


In [ ]:
weather_df = weather_df.withColumn('prcp', weather_df['prcp'].cast(IntegerType()))
weather_df = weather_df.withColumn('snow', weather_df['snow'].cast(IntegerType()))
weather_df = weather_df.withColumn('pres', weather_df['pres'].cast(IntegerType()))
weather_df = weather_df.withColumn('tavg', weather_df['tavg'].cast(IntegerType()))
weather_df = weather_df.withColumn('tmax', weather_df['tmax'].cast(IntegerType()))
weather_df = weather_df.withColumn('tmin', weather_df['tmin'].cast(IntegerType()))
weather_df = weather_df.withColumn('tsun', weather_df['tsun'].cast(IntegerType()))
weather_df = weather_df.withColumn('wdir', weather_df['wdir'].cast(IntegerType()))
weather_df = weather_df.withColumn('wpgt', weather_df['wpgt'].cast(IntegerType()))
weather_df = weather_df.withColumn('wspd', weather_df['wspd'].cast(IntegerType()))

In [ ]:
weather_df=weather_df.join(aircraft_df.select(['CADORS Number']),['CADORS Number'])
weather_df=weather_df.dropDuplicates()

In [ ]:
print(weather_df.count())
print(weather_df.select(['CADORS Number']).distinct().count())

212176
212176


CLUSTERING FOR ONLY ACCIDENTS

In [ ]:
def merge(x):
  if len(x)==0:
    return 'NA'
  temp=''
  x=sorted(x)
  for i in range(len(x)):
    temp+=x[i]
  return temp
  
def con(x):
    if x=='':
        return 'NA'
    return x

In [ ]:
transform_udf=udf(merge,StringType())
makeNA_udf = udf(con,StringType())

#picking features from df( Main Data)
featureset1=aircraft_df.select(['CADORS Number','Occurrence Category','hour','GR','Occurrence Event Information','month']).dropDuplicates()
featureset1=featureset1.withColumn('Occurrence Event Information_merged',transform_udf(col('Occurrence Event Information'))).withColumn('Occurrence Category_merged',transform_udf(col('Occurrence Category')))
featureset1=featureset1.withColumn('Occurrence Event Information_merged',makeNA_udf(col('Occurrence Event Information_merged'))).withColumn('Occurrence Category_merged',makeNA_udf(col('Occurrence Category_merged')))
print("FEATURES 1\n")
print(featureset1.count())
print(featureset1.head(5))


FEATURES 1

212176
[Row(CADORS Number='1994C0090', Occurrence Category=['System/component failure or malfunction [powerplant]', 'Other'], hour=16, GR='Manitoba', Occurrence Event Information=[], month=3, Occurrence Event Information_merged='NA', Occurrence Category_merged='OtherSystem/component failure or malfunction [powerplant]'), Row(CADORS Number='1994W0678', Occurrence Category=['System/component failure or malfunction [non-powerplant]'], hour=1, GR='Yukon', Occurrence Event Information=[], month=8, Occurrence Event Information_merged='NA', Occurrence Category_merged='System/component failure or malfunction [non-powerplant]'), Row(CADORS Number='1995O0125', Occurrence Category=['System/component failure or malfunction [non-powerplant]'], hour=20, GR='Ontario', Occurrence Event Information=[], month=3, Occurrence Event Information_merged='NA', Occurrence Category_merged='System/component failure or malfunction [non-powerplant]'), Row(CADORS Number='1995P0129', Occurrence Category=[

In [ ]:
#features from aircraft data
# aircraft_information1=aircraft_information1.join(df.select('CADORS Number'),['CADORS Number'])
featureset2 = aircraft_df.groupBy('CADORS Number').agg(collect_list('flight_rule').alias('flight_rule'),collect_list('year_built').alias('year_built'),collect_list('amateur_built').alias('amateur_built'),collect_list('gear_type').alias('gear_type'),collect_list('phase_of_flight').alias('phase_of_flight'),collect_list('damage').alias('damage'),collect_list('operator_type').alias('operator_type'))
featureset2=featureset2.withColumn('flight_rule_merged',transform_udf(col('flight_rule'))).withColumn('year_built_merged',transform_udf(col('year_built'))).withColumn('amateur_built_merged',transform_udf(col('amateur_built'))).withColumn('gear_type_merged',transform_udf(col('gear_type'))).withColumn('phase_of_flight_merged',transform_udf(col('phase_of_flight'))).withColumn('damage_merged',transform_udf(col('damage'))).withColumn('operator_type_merged',transform_udf(col('operator_type')))
featureset2=featureset2.withColumn('flight_rule_merged',makeNA_udf(col('flight_rule_merged'))).withColumn('year_built_merged',makeNA_udf(col('year_built_merged'))).withColumn('amateur_built_merged',makeNA_udf(col('amateur_built_merged'))).withColumn('gear_type_merged',makeNA_udf(col('gear_type_merged'))).withColumn('phase_of_flight_merged',makeNA_udf(col('phase_of_flight_merged'))).withColumn('damage_merged',makeNA_udf(col('damage_merged'))).withColumn('operator_type_merged',makeNA_udf(col('operator_type_merged')))
print("FEATURES 2\n")
print(featureset2.count())
print(featureset2.head(5))


FEATURES 2

212176
[Row(CADORS Number='1993C0067', flight_rule=['NA'], year_built=['1982'], amateur_built=['No'], gear_type=['NA'], phase_of_flight=['Unknown'], damage=['Unknown'], operator_type=['Commercial'], flight_rule_merged='NA', year_built_merged='1982', amateur_built_merged='No', gear_type_merged='NA', phase_of_flight_merged='Unknown', damage_merged='Unknown', operator_type_merged='Commercial'), Row(CADORS Number='1993C0072', flight_rule=['NA'], year_built=['1967'], amateur_built=['No'], gear_type=['NA'], phase_of_flight=['Unknown'], damage=['Unknown'], operator_type=['Commercial'], flight_rule_merged='NA', year_built_merged='1967', amateur_built_merged='No', gear_type_merged='NA', phase_of_flight_merged='Unknown', damage_merged='Unknown', operator_type_merged='Commercial'), Row(CADORS Number='1993C0084', flight_rule=['NA'], year_built=['NA'], amateur_built=['No'], gear_type=['NA'], phase_of_flight=['Unknown'], damage=['Unknown'], operator_type=['NA'], flight_rule_merged='NA', 

In [ ]:
# #features from weather data
featureset3=weather_df.select(['CADORS Number','prcp','snow']).dropDuplicates()
featureset3=featureset3.dropna(how='any')
print("FEATURES 3\n")
print(featureset3.count())
print(featureset3.head(5))

FEATURES 3

158499
[Row(CADORS Number='1995C0405', prcp=0, snow=250), Row(CADORS Number='1995O0273', prcp=0, snow=0), Row(CADORS Number='1995P0141', prcp=0, snow=270), Row(CADORS Number='1995Q0310', prcp=0, snow=200), Row(CADORS Number='1995Q0689', prcp=0, snow=320)]


In [ ]:
## Final data for ML
ff=featureset1.join(featureset2,['CADORS Number'],how='inner')
ff=ff.join(featureset3,['CADORS Number'],how='inner')
print(ff.count())
print(ff.head(5))

158499
[Row(CADORS Number='1993C0072', Occurrence Category=['System/component failure or malfunction [powerplant]'], hour=18, GR='Ontario', Occurrence Event Information=[], month=2, Occurrence Event Information_merged='NA', Occurrence Category_merged='System/component failure or malfunction [powerplant]', flight_rule=['NA'], year_built=['1967'], amateur_built=['No'], gear_type=['NA'], phase_of_flight=['Unknown'], damage=['Unknown'], operator_type=['Commercial'], flight_rule_merged='NA', year_built_merged='1967', amateur_built_merged='No', gear_type_merged='NA', phase_of_flight_merged='Unknown', damage_merged='Unknown', operator_type_merged='Commercial', prcp=1, snow=760), Row(CADORS Number='1993C0151', Occurrence Category=['Collision with obstacle(s) during take-off or landing whilst airborne.', 'Loss of control - ground'], hour=18, GR='Ontario', Occurrence Event Information=[], month=5, Occurrence Event Information_merged='NA', Occurrence Category_merged='Collision with obstacle(s) du

In [ ]:
######################### ML DATA ####################################
finalMLData=ff

In [ ]:
for k in range(9,10):
  # str_indexer = StringIndexer(inputCols=['Canadian Aerodrome ID:','Country:','TC Region:','Day Or Night:','World Area:','Occurrence Type:','Province:','Occurrence Event Information_merged','Occurrence Category_merged','aircraft_category_merged','flight_rule_merged','cars_subpart_merged'], outputCols=['Canadian Aerodrome ID1:','Country1:','TC Region1:','Day Or Night1:','World Area1:','Occurrence Type1:','Province1:','Occurrence Event Information_merged1','Occurrence Category_merged1','aircraft_category_merged1','flight_rule_merged1','cars_subpart_merged1'])
  # m=str_indexer.fit(finalfeatures)
  # td=m.transform(finalfeatures)
  # print(td.head())
  # print(str_indexer.transform(finalfeatures)).head()
  str_indexer = StringIndexer(inputCols=['Occurrence Category_merged','GR','Occurrence Event Information_merged','flight_rule_merged','year_built_merged','amateur_built_merged','gear_type_merged','phase_of_flight_merged','damage_merged','operator_type_merged'], outputCols=['Occurrence Category_merged1','GR1','Occurrence Event Information_merged1','flight_rule_merged1','year_built_merged1','amateur_built_merged1','gear_type_merged1','phase_of_flight_merged1','damage_merged1','operator_type_merged1'],handleInvalid="keep")
  assembler = VectorAssembler(inputCols=['hour','month','Occurrence Category_merged1','GR1','Occurrence Event Information_merged1','flight_rule_merged1','year_built_merged1','amateur_built_merged1','gear_type_merged1','phase_of_flight_merged1','phase_of_flight_merged1','damage_merged1','operator_type_merged1','operator_type_merged1','prcp','snow'], outputCol="features")
  # print(assembler.transform(finalfeatures).head(5))
  kmeans = KMeans(k=k,seed=1,featuresCol="features",predictionCol="prediction")
  clustering_pipeline = Pipeline(stages=[str_indexer,assembler,kmeans])
  model=clustering_pipeline.fit(finalMLData)
  predictions = model.transform(finalMLData)
  # Evaluate clustering by computing Silhouette score
  evaluator = ClusteringEvaluator()
  silhouette = evaluator.evaluate(predictions)
  print("Silhouette for k= ",k," with squared euclidean distance = " + str(silhouette),'\n')
  predictions.show()
# kmeans = KMeans(k=3,seed=1,featuresCol="features",predictionCol="prediction")
# clustering_pipeline = Pipeline(stages=[str_indexer,assembler,kmeans])
# model=clustering_pipeline.fit(finalfeatures)
# predictions = model.transform(finalfeatures)

Silhouette for k=  9  with squared euclidean distance = 0.8455568938187714 

+-------------+--------------------+----+--------+----------------------------+-----+-----------------------------------+--------------------------+-----------+----------+-------------+---------+---------------+-------------+-------------+------------------+-----------------+--------------------+----------------+----------------------+-------------+--------------------+----+----+---------------------------+----+------------------------------------+-------------------+------------------+---------------------+-----------------+-----------------------+--------------+---------------------+--------------------+----------+
|CADORS Number| Occurrence Category|hour|      GR|Occurrence Event Information|month|Occurrence Event Information_merged|Occurrence Category_merged|flight_rule|year_built|amateur_built|gear_type|phase_of_flight|       damage|operator_type|flight_rule_merged|year_built_merged|amateur_built_merged|g

In [ ]:
val_data.printSchema()

root
 |-- CADORS Number: string (nullable = true)
 |-- prediction: integer (nullable = false)
 |-- Fatalities:: long (nullable = true)
 |-- GR: string (nullable = true)
 |-- Injuries:: long (nullable = true)
 |-- Occurrence Category: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- Occurrence Date:: string (nullable = true)
 |-- Occurrence Event Information: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- Occurrence Location:: string (nullable = true)
 |-- Occurrence Type:: string (nullable = true)
 |-- Province:: string (nullable = true)
 |-- aircraft_category: string (nullable = true)
 |-- amateur_built: string (nullable = true)
 |-- damage: string (nullable = true)
 |-- flight_rule: string (nullable = true)
 |-- gear_type: string (nullable = true)
 |-- hour: long (nullable = true)
 |-- label: long (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- make: string (nullable = tr

In [ ]:
val_data.groupby('prediction').agg(sum(col('Fatalities:')),sum(col('Injuries:'))).show()

+----------+----------------+--------------+
|prediction|sum(Fatalities:)|sum(Injuries:)|
+----------+----------------+--------------+
|         1|             101|           386|
|         6|               4|            46|
|         3|               2|            16|
|         5|              18|            33|
|         4|               0|             8|
|         8|             197|           601|
|         7|              57|           137|
|         2|              34|             4|
|         0|             778|          1551|
+----------+----------------+--------------+



In [ ]:
predictions = spark.read.json("/content/gdrive/MyDrive/cadors_KM_9/*")
aircraft_df = spark.read.json('/content/gdrive/MyDrive/cadorsdata_aircraft/*')

In [ ]:
sample=predictions.select(['CADORS Number','GR1','Occurrence Category_merged1','Occurrence Event Information_merged1','amateur_built_merged1','damage_merged1','flight_rule_merged1','gear_type_merged1','hour','month','operator_type_merged1','phase_of_flight_merged1','prcp','snow','year_built_merged1','prediction'])

In [ ]:
sample.write.option('header',True).csv('/content/gdrive/MyDrive/cadorsdata_KM_clustering_sample/*')

In [ ]:
val_data=predictions.select(['CADORS Number','prediction']).join(aircraft_df,['CADORS Number'])
val_data=val_data.dropDuplicates()

In [ ]:
val_data=val_data.select(['CADORS Number','Injuries:','Fatalities:','prediction']).dropDuplicates()
print(val_data.count())
print(val_data.select('CADORS Number').distinct().count())

158499
158499


In [ ]:
sample_clustering=val_data.sample(withReplacement=False, fraction=0.01).write.option('header',True).csv('/content/gdrive/MyDrive/cadorsdata_K9_sample')

In [ ]:
# val_data.show()
temp=val_data.groupby('prediction').agg(countDistinct('CADORS Number'),sum('Injuries:'),sum('Fatalities:'),sum('Injuries:')+sum('Fatalities:').alias('totalharm'),((sum('Injuries:')+sum('Fatalities:'))*100/countDistinct('CADORS Number')).alias('harmppercent'))
temp.sort(['harmppercent'],ascending = [False]).write.option('header',True).csv('/content/gdrive/MyDrive/cadorsdata_K9_result1/')
